# Causal Inference: Effect of Prior Circulatory Diagnosis on Readmission

**Objective:** Investigate the causal effect of having a prior diagnosis related to the 'Circulatory System' (based on ICD-9 chapters) on the risk of 30-day hospital readmission, controlling for patient age.

**Method:** Propensity Score Matching (PSM) using Nearest Neighbors.

**Key Steps:**
1. Load Data
2. Define Treatment, Outcome, and Covariates
3. Estimate Propensity Scores
4. Perform Matching
5. Check Covariate Balance
6. Estimate Treatment Effect (ATT)
7. Discuss Assumptions and Limitations

## 1. Load Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Add project root for utils
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils import load_config, get_data_path

# Load config and data path
config = load_config()
data_path = get_data_path('processed', 'combined_features', config)

# Load data
try:
    df = pd.read_csv(data_path)
    print(f"Data loaded successfully from {data_path}. Shape: {df.shape}")
    # Display first few rows and columns
    pd.set_option('display.max_columns', 100)
    display(df.head())
except FileNotFoundError:
    print(f"Error: Data file not found at {data_path}")
except Exception as e:
    print(f"Error loading data: {e}")

## 2. Define Treatment, Outcome, and Covariates

- **Treatment (T):** Binary indicator (1 if patient has a prior diagnosis in the 'Diseases of the Circulatory System' ICD-9 chapter (390-459), 0 otherwise). We need to identify the relevant diagnosis columns created during feature engineering.
- **Outcome (Y):** `readmission_30day` (binary).
- **Covariates (X):** `age` (continuous). We might add more later (e.g., gender, other comorbidities) but start simple.

In [ ]:
# Identify diagnosis columns related to circulatory system (assuming they exist)
# This depends heavily on how diagnosis features were created.
# Example: Assuming one-hot encoded columns like 'diag_cat_circulatory_system'
circulatory_diag_col = 'diag_cat_diseases_of_the_circulatory_system' # Placeholder - ADJUST BASED ON ACTUAL FEATURE NAME

if circulatory_diag_col not in df.columns:
    print(f"Error: Assumed treatment column '{circulatory_diag_col}' not found.")
    print("Available columns:", df.columns.tolist())
    # You might need to reconstruct this feature based on raw diagnosis codes if not present
    # Or adjust the column name based on feature engineering output.
    treatment_defined = False
else:
    df['treatment'] = df[circulatory_diag_col].apply(lambda x: 1 if x > 0 else 0) # Ensure binary
    treatment_defined = True
    print(f"Treatment column ('{circulatory_diag_col}') found and processed.")

# Define outcome
outcome_col = 'readmission_30day'
if outcome_col not in df.columns:
    print(f"Error: Outcome column '{outcome_col}' not found.")
    outcome_defined = False
else:
    df['outcome'] = df[outcome_col]
    outcome_defined = True
    print(f"Outcome column ('{outcome_col}') found.")

# Define covariates
covariate_cols = ['age'] # Start with age
covariates_defined = all(c in df.columns for c in covariate_cols)
if not covariates_defined:
    print(f"Error: Not all covariate columns ({covariate_cols}) found.")
else:
     print(f"Covariate columns ({covariate_cols}) found.")

# Filter data to necessary columns and drop rows with NaNs in these specific columns
required_cols = ['treatment', 'outcome'] + covariate_cols
analysis_df = None
if treatment_defined and outcome_defined and covariates_defined:
    analysis_df = df[required_cols].copy()
    initial_rows = len(analysis_df)
    analysis_df.dropna(inplace=True)
    final_rows = len(analysis_df)
    print(f"\nAnalysis DataFrame created. Dropped {initial_rows - final_rows} rows due to NaNs in required columns.")
    print(f"Final shape for analysis: {analysis_df.shape}")
    display(analysis_df.head())
    display(analysis_df.describe())
    print("\nTreatment group counts:")
    print(analysis_df['treatment'].value_counts())
else:
    print("\nCannot proceed with analysis due to missing columns.")

## 3. Estimate Propensity Scores

We estimate the probability of receiving the treatment (having a prior circulatory diagnosis) given the covariates (age). We'll use Logistic Regression.

In [ ]:
if analysis_df is not None:
    # Define features (X) and target (T) for propensity score model
    X = analysis_df[covariate_cols]
    T = analysis_df['treatment']

    # Add constant for statsmodels
    X_const = sm.add_constant(X)

    # Fit logistic regression model
    # Using statsmodels for more detailed output, could use sklearn too
    logit_model = sm.Logit(T, X_const)
    try:
        ps_model_results = logit_model.fit(disp=0) # disp=0 suppresses convergence messages
        print(ps_model_results.summary())

        # Predict propensity scores
        analysis_df['propensity_score'] = ps_model_results.predict(X_const)

        # Visualize propensity score distribution
        plt.figure(figsize=(10, 6))
        sns.histplot(data=analysis_df, x='propensity_score', hue='treatment', kde=True, stat='density', common_norm=False)
        plt.title('Propensity Score Distribution by Treatment Group')
        plt.xlabel('Estimated Propensity Score')
        plt.show()

        print("\nPropensity score summary statistics:")
        print(analysis_df.groupby('treatment')['propensity_score'].describe())
        propensity_scores_estimated = True
    except Exception as e:
        print(f"Error fitting propensity score model: {e}")
        propensity_scores_estimated = False
else:
    print("Analysis DataFrame not available. Skipping propensity score estimation.")
    propensity_scores_estimated = False

**Common Support Check:** Examine the histograms above. We need overlap in the propensity scores between the treated and control groups. If there are regions where only treated or only control units exist, matching may not be reliable for those units (lack of common support).

## 4. Perform Matching

We'll use 1-to-1 nearest neighbor matching based on the propensity score *without* replacement.

In [ ]:
if analysis_df is not None and propensity_scores_estimated:
    treated = analysis_df[analysis_df['treatment'] == 1]
    control = analysis_df[analysis_df['treatment'] == 0]

    if treated.empty or control.empty:
        print("Error: One or both treatment/control groups are empty. Cannot perform matching.")
        matching_performed = False
    else:
        # Use NearestNeighbors from sklearn
        nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
        nn.fit(control[['propensity_score']])

        # Find the nearest control neighbor for each treated unit
        distances, indices = nn.kneighbors(treated[['propensity_score']])

        # Create matched control group
        matched_control_indices = control.iloc[indices.flatten()].index
        matched_control = control.loc[matched_control_indices]

        # Combine treated and matched controls into a final matched dataset
        matched_df = pd.concat([treated, matched_control], ignore_index=True)

        print(f"Matching complete. Matched dataset shape: {matched_df.shape}")
        print("\nMatched dataset treatment counts:")
        print(matched_df['treatment'].value_counts())
        matching_performed = True
else:
    print("Propensity scores not estimated or data missing. Skipping matching.")
    matching_performed = False

## 5. Check Covariate Balance

After matching, the distribution of covariates (age) should be similar between the treated and matched control groups. We check this using the Standardized Mean Difference (SMD).

In [ ]:
def calculate_smd(group1, group2, var):
    """Calculates the standardized mean difference for a variable."""
    mean1, mean2 = group1[var].mean(), group2[var].mean()
    std1, std2 = group1[var].std(), group2[var].std()
    # Pooled standard deviation
    pooled_std = np.sqrt((std1**2 + std2**2) / 2)
    if pooled_std == 0:
        return np.nan # Avoid division by zero
    return (mean1 - mean2) / pooled_std

if matching_performed:
    balance_results = {}
    
    # Balance before matching
    print("--- Balance Before Matching ---")
    treated_before = analysis_df[analysis_df['treatment'] == 1]
    control_before = analysis_df[analysis_df['treatment'] == 0]
    for cov in covariate_cols:
        smd_before = calculate_smd(treated_before, control_before, cov)
        balance_results[cov] = {'SMD Before': smd_before}
        print(f"{cov}: SMD = {smd_before:.4f}")

    # Balance after matching
    print("\n--- Balance After Matching ---")
    treated_after = matched_df[matched_df['treatment'] == 1]
    control_after = matched_df[matched_df['treatment'] == 0]
    for cov in covariate_cols:
        smd_after = calculate_smd(treated_after, control_after, cov)
        balance_results[cov]['SMD After'] = smd_after
        print(f"{cov}: SMD = {smd_after:.4f}")

    # Generally, SMD < 0.1 indicates good balance
    balance_df = pd.DataFrame(balance_results).T
    display(balance_df)
    balance_checked = True
else:
    print("Matching not performed. Skipping balance check.")
    balance_checked = False

## 6. Estimate Treatment Effect (ATT)

We estimate the Average Treatment Effect on the Treated (ATT) by comparing the outcome (`readmission_30day`) between the treated group and the matched control group.

In [ ]:
if matching_performed and balance_checked:
    # Simple difference in means for the outcome in the matched sample
    mean_outcome_treated = matched_df[matched_df['treatment'] == 1]['outcome'].mean()
    mean_outcome_control = matched_df[matched_df['treatment'] == 0]['outcome'].mean()

    att_estimate = mean_outcome_treated - mean_outcome_control

    print(f"Mean outcome (readmission) for Treated group (matched): {mean_outcome_treated:.4f}")
    print(f"Mean outcome (readmission) for Control group (matched): {mean_outcome_control:.4f}")
    print(f"\nEstimated ATT (Difference in Means): {att_estimate:.4f}")

    # Optional: Use regression on the matched dataset for potentially more robust estimation
    # Y ~ T + X (using matched_df)
    X_matched = matched_df[covariate_cols]
    T_matched = matched_df['treatment']
    Y_matched = matched_df['outcome']
    X_matched_const = sm.add_constant(pd.concat([T_matched, X_matched], axis=1))

    try:
        ols_model = sm.OLS(Y_matched, X_matched_const)
        ols_results = ols_model.fit()
        print("\n--- OLS Regression on Matched Data ---")
        print(ols_results.summary())
        att_regression = ols_results.params['treatment']
        print(f"\nEstimated ATT (from OLS coefficient): {att_regression:.4f}")
    except Exception as e:
        print(f"\nError fitting OLS model on matched data: {e}")

else:
    print("Matching or balance check failed. Skipping ATT estimation.")

## 7. Discussion: Assumptions and Limitations

**Assumptions:**
1.  **Stable Unit Treatment Value Assumption (SUTVA):** An individual's outcome is only affected by their own treatment status, not the treatment status of others, and there's only one version of the treatment.
2.  **Ignorability / Unconfoundedness:** Conditional on the observed covariates (age), the treatment assignment is independent of the potential outcomes. This is a strong assumption – it means we've measured and controlled for *all* factors that influence both the likelihood of having a circulatory diagnosis *and* the likelihood of readmission. This is likely **violated** here as 'age' is insufficient. We would need to include many more clinical variables (other comorbidities, severity scores, lab values, etc.) to make this more plausible.
3.  **Positivity / Common Support:** For every combination of covariates, there is a non-zero probability of being both treated and untreated. We checked this visually with the propensity score overlap.

**Limitations:**
- **Observational Data:** We cannot definitively prove causation, only estimate association while controlling for observed confounders.
- **Unmeasured Confounding:** The Ignorability assumption is hard to satisfy. Factors not included in our covariates (e.g., socioeconomic status, specific procedures, medication adherence, unmeasured disease severity) could bias the results.
- **Model Dependence:** The results depend on the correctness of the propensity score model (logistic regression).
- **Matching Quality:** Nearest neighbor matching can sometimes result in poor matches if good controls aren't available (check distances if needed). It also discards unmatched units, potentially changing the population the results generalize to.
- **Simplified Analysis:** This is a basic example using only age. A real-world analysis would require careful selection and inclusion of many more relevant covariates and potentially more advanced methods (e.g., Doubly Robust Estimation).